In [20]:
module type BOVINE = 
  sig
    type t
    val cow : t
    val equal : t -> t -> bool
    val to_string : t -> string
  end ;;

module Cow : BOVINE =
  struct
    type t = string
    let cow = "Moo"
    let equal x y = ( x = y)
    let to_string x = x
  end ;;

let c1 = Cow.cow ;;
let _ = Printf.printf "Our first cow says: %s\n" (Cow.to_string c1) ;; flush stdout ;;

module type BOVINE =
  sig
    type t
    val cow : t
    val equal : t -> t -> bool
    val to_string : t -> string
  end


module Cow : BOVINE


val c1 : Cow.t = <abstr>


- : unit = ()


Our first cow says: Moo


- : unit = ()


# Množice

Ogledali si bomo dve implementaciji podatkovnega tipa [množica](https://en.wikipedia.org/wiki/Set_(abstract_data_type)). V jeziku OCaml množico opišemo z naslednjo signaturo:
<a id='module_set_cell'></a>

In [3]:
(* Signatura množica *)
type order = Less | Equal | Greater

module type SET =
sig
    type element
    val cmp : element -> element -> order
    type set
    val empty : set
    val member : element -> set -> bool
    val add : element -> set -> set
    val remove : element -> set -> set
    val to_list : set -> element list
    val fold : ('a -> element -> 'a) -> 'a -> set -> 'a
end

type order = Less | Equal | Greater


module type SET =
  sig
    type element
    val cmp : element -> element -> order
    type set
    val empty : set
    val member : element -> set -> bool
    val add : element -> set -> set
    val remove : element -> set -> set
    val to_list : set -> element list
    val fold : ('a -> element -> 'a) -> 'a -> set -> 'a
  end


In [13]:
(* Funkcija za primerjavo vrednost *)
let ocaml_cmp x y =
    let c = Stdlib.compare x y in
    if c < 0 then Less
    else if c > 0 then Greater
    else Equal ;;

(* Definiramo strukturu IntListSet, ki elemente mnozice hrani v seznami, tip elementov naj bo celo stevilo, za primerjavo pa uporabimo ocaml_cmp *)
module IntListSet : SET with type element = int =
struct
    type element = int
    let cmp = ocaml_cmp
    type set = int list
    let empty = []
    let rec member x = function
        | [] -> false
        | y :: ys -> (x = y) || member x ys
    let add x ys = if member x ys then ys else (x :: ys)
    let rec remove x = function
        | [] -> []
        | y :: ys -> if x = y then ys else y :: remove x ys
    (* Convert set to list, but our set is alredy list, so just return it*)
    let to_list ys = ys 

    (* Apply the function f to the accumulator x and the every element in the array *)
    let rec fold f x = function
        | [] -> x
        | y :: ys -> fold f (f x y) ys

end ;;

(* Add some elements to the set *)
let s = IntListSet.add 3 (IntListSet.add 2 (IntListSet.add 1 IntListSet.empty)) ;; flush stdout;;
let _ = Printf.printf "Is 2 a member of the set? %b\n" (IntListSet.member 2 s) ;; flush stdout;;
let _ = Printf.printf "Is 4 a member of the set? %b\n" (IntListSet.member 4 s) ;; flush stdout;;
let s = IntListSet.remove 1 s ;; flush stdout;;
let _ = Printf.printf "Is 1 a member of the set? %b\n" (IntListSet.member 1 s) ;; flush stdout;;

(* Convert the set to a list and print it *)
let list = IntListSet.to_list s ;;
let _ = Printf.printf "The set converted to a list is: %s\n" 
  (String.concat ", " (List.map string_of_int list)) ;;
flush stdout;;

(* Fold the sum of all elements with 2*)
let sum = IntListSet.fold (fun x y -> x + y + 2) 0 s ;;
let _ = Printf.printf "The sum of all elements in the set is: %d\n" sum ;; flush stdout;; (* [3, 2] -- > 9*)

val ocaml_cmp : 'a -> 'a -> order = <fun>


module IntListSet :
  sig
    type element = int
    val cmp : element -> element -> order
    type set
    val empty : set
    val member : element -> set -> bool
    val add : element -> set -> set
    val remove : element -> set -> set
    val to_list : set -> element list
    val fold : ('a -> element -> 'a) -> 'a -> set -> 'a
  end


val s : IntListSet.set = <abstr>


- : unit = ()


- : unit = ()


...

- : unit = ()


val sum : int = 9


- : unit = ()


The sum of all elements in the set is: 9


- : unit = ()


## Generični `ListSet`

Definicijo `IntListSet` spremenite v funktor `ListSet`, ki sprejme strukturo, ki zadošča signaturi

In [ ]:
module type ORDERED =
    sig
        type t
        val cmp : t -> t -> order
    end

ter vrne implementacijo množic s seznami. Nastavek je naslednji:

    module ListSet(M : ORDERED) : SET with type element = M.t =
      struct
         ⋮
      end

Primer uporabe:

    # module S = ListSet(struct type t = string  let cmp = ocaml_cmp end) ;;
    module S :
      sig
        type element = string
        val cmp : element -> element -> order
        type set
        val empty : set
        val member : element -> set -> bool
        val add : element -> set -> set
        val remove : element -> set -> set
        val to_list : set -> element list
        val fold : ('a -> element -> 'a) -> 'a -> set -> 'a
      end
    # let s = S.add "foo" (S.add "bar" S.empty) ;;
    val s : S.set = <abstr>
    # S.member "foo" s ;;
    - : bool = true

Zgledujte se po implementaciji prioritetne vrste `ListQueue` s predavanj.
<a id='ListSet'></a>

In [ ]:
module ListSet(M : ORDERED) : SET with type element = M.t =
struct
    (* dopolni *)
end
  
module S = ListSet(struct type t = string  let cmp = ocaml_cmp end) ;;
let s = S.add "foo" (S.add "bar" S.empty) ;;
S.member "foo" s ;;

## Primerjava učinkovitosti `ListSet` in `AVLSet`

Modul `AVLSet` implementira funktor, ki tako kot `ListSet` sprejme strukturo s signaturo `ORDERED` in vrne
implementacijo množic z AVL drevesi, glej datoteko `avlset.ml`.
<a id='AVLSet'></a>

In [ ]:
(* datoteka avlset.ml *)
module AVLSet(M : ORDERED) : SET with type element = M.t =
  struct
    type element = M.t
    let cmp = M.cmp

    type set = Empty | Node of element * int * set * set

    let empty = Empty

    let height = function
      | Empty -> 0
      | Node (_, h, _, _) -> h

    let leaf v = Node (v, 1, Empty, Empty)

    let node (v, l, r) = Node (v, 1 + max (height l) (height r), l, r)

    let rec member x = function
      | Empty -> false
      | Node (y, _, l, r) ->
         begin
           match cmp x y with
           | Equal -> true
           | Less -> member x l
           | Greater -> member x r
         end

    let rec to_list = function
      | Empty -> []
      | Node (x, _, l, r) -> to_list l @ [x] @ to_list r

    let rotateLeft = function
      | Node (x, _, a, Node (y, _, b, c)) -> node (y, node (x, a, b), c)
      | t -> t

    (* alternativni zapis s case *)
    let rotateRight = function
      | Node (y, _, Node (x, _, a, b), c) -> node (x, a, node (y, b, c))
      | t -> t

    let imbalance = function
      | Empty -> 0
      | Node (_, _, l, r) -> height l - height r

    let balance = function
      | Empty -> Empty
      | Node (x, _, l, r) as t ->
         begin
           match imbalance t with
           | 2 ->
              (match imbalance l with
               | -1 -> rotateRight (node (x, rotateLeft l, r))
               | _ -> rotateRight t)
           | -2 ->
              (match imbalance r with
               | 1 -> rotateLeft (node (x, l, rotateRight r))
               | _ -> rotateLeft t)
           | _ -> t
         end

    let rec add x = function
      | Empty -> leaf x
      | Node (y, _, l, r) as t ->
         begin
           match cmp x y with
           | Equal -> t
           | Less -> balance (node (y, add x l, r))
           | Greater -> balance (node (y, l, add x r))
         end

    let rec remove x = function
      | Empty -> Empty
      | Node (y, _, l, r) ->
         let rec removeSuccessor = function
           | Empty -> assert false
           | Node (x, _, Empty, r) -> (r, x)
           | Node (x, _, l, r) ->
              let (l', y) = removeSuccessor l in
              (balance (node (x, l', r)), y)
         in
         match cmp x y with
         | Less -> balance (node (y, remove x l, r))
         | Greater -> balance (node (y, l, remove x r))
         | Equal ->
            begin match (l, r) with
            | (_, Empty) -> l
            | (Empty, _) -> r
            | _ ->
               let (r', y') = removeSuccessor r in
               balance (node (y', l, r'))
            end

    let rec fold f x = function
      | Empty -> x
      | Node (y, _, l, r) -> fold f (f (fold f x l) y) r

  end

Preverimo, da je `AVLSet` res učinkovitejši od `ListSet`. Najprej definiramo dve
implementaciji množic celih števil, eno s seznami in eno z AVL drevesi:

In [ ]:
module SL = ListSet (struct type t = int let cmp = ocaml_cmp end)
module SA = AVLSet (struct type t = int let cmp = ocaml_cmp end)

### Množica `{1, 2, ..., n}`

Definirajte funkcijo `add_list n`, ki vrne množico (implementirano s seznami) števil od `1` do `n`. Primer uporabe:

    # SL.to_list (add_list 10) ;;
    - : SL.element list = [10; 9; 8; 7; 6; 5; 4; 3; 2; 1]

In [ ]:
(* let rec add_list = … *)
SL.to_list (add_list 10)

Na enak način definirajte še funkcijo `add_avl`, ki množico implementira z AVL drevesi:

    # SA.to_list (add_avl 10) ;;
    - : SA.element list = [1; 2; 3; 4; 5; 6; 7; 8; 9; 10]

In [ ]:
(* let rec add_avl = … *)
SA.to_list (add_avl 10)

### Čas izvajanja

Podana je funkcija `time`, ki izmeri čas izvajanja dane funkcije `f`:

   

In [ ]:
let time f =
    let start = Sys.time () in
    f () ;
    let stop = Sys.time () in
    stop -. start

Funkcija `f` je »thunk«, torej sprejme le argument tipa `unit`. Čas, ki ga OCaml potrebuje, da sešteje vsa števila od 1 do 10000000:

    # time (fun () -> let s = ref 0 in for i = 1 to 10000000 do s := !s + i done; !s) ;;
    - : float = 0.17635

Izmerite, koliko časa traja vstavljanje velikega števila elementov v množico.
Primerjajte implementaciji `SL` in `SA`.

In [ ]:
(* time (fun () -> … ) ;;
time … *)

## Fold

Funkcija `fold` sprejme

* funkcijo `f` dveh argumentov,
* začetno vrednost `z` in
* množico `m = {x₁, x₂, …, xᵢ}`

ter izračuna rezultat tako, da s pomočjo `f` elemente enega za drugim kombinira s trenutno vrednostjo:

    f (… (f (f z x₁) x₂) …) xᵢₙ

Primer, kjer je funkcija `f` kar sešetvane, začetna vrednost `z` je `0` in množica `m` je `{1,2,3,4}`:

    # SL.fold ( + ) 0 (add_list 4) ;;
    - : int = 10

Signaturi [`SET`](#module_set_cell) dodajte funkcijo `fold`:

    val fold : ('a -> element -> 'a) -> 'a -> set -> 'a

in jo implementirajte v funktorjih [`ListSet`](#ListSet) in [`AVLSet`](#AVLSet).

## Unija, filter in presek

Definirajte funktor `SetOps`, ki sprejme strukturo s signaturo `SET` in
implementira funkcije `union`, `filter` ter `intersection`. Nastavek za rešitev:

    module SetOps(S : SET) =
    struct
       let union a b = …
       let filter p a = …
       let intersection a b = …
    end

Unijo in presek poznamo, funkcija `filter` pa sprejme predikat (funkcijo, ki
vrača vrednosti tipa `bool`) in množico, ter novo množico tistih elementov, ki
zadoščajo predikatu. Pri definiciji funkcij si pomagajte s funkcijo `fold`.

Primer:

    # module SA_ops = SetOps(SA) ;;
    module SA_ops :
      sig
        val union : SA.set -> SA.set -> SA.set
        val filter : (SA.element -> bool) -> SA.set -> SA.set
        val intersection : SA.set -> SA.set -> SA.set
      end
    # SA.to_list (SA_ops.filter (fun x -> x > 10) (add_avl 20)) ;;
    - : SA.element list = [11; 12; 13; 14; 15; 16; 17; 18; 19; 20]

In [ ]:
module SetOps(S : SET) =
  struct
    (* dopolni *)
  end

module SA_ops = SetOps(SA)
;;
SA.to_list (SA_ops.filter (fun x -> x > 10) (add_avl 20))